# SBV2学習用のGoogle Colabスクリプト
公式の下記の実装を使わせていただいております。

https://github.com/litagin02/Style-Bert-VITS2/blob/master/colab.ipynb

「ランタイム」→「ランタイムのタイプを変更」

から、「T4 GPU」に変更してから、「Shift+Enter」で上からセルを実行してください。

# Google Driveのマウント
認証が入るので、最初のセルに置くのをオススメします。

In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# 必要パッケージのインストール

In [ ]:
# このセルを実行して環境構築してください。
# エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが「キャンセル」を選択して続行してください。

import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements-colab.txt
!python initialize.py --skip_default_models

# 各種パスやモデル名の指定

In [ ]:
model_name = "amitaro"

# Google Driveでのパスを特定する
import glob
import os
drive_path = os.path.dirname(glob.glob('/content/drive/MyDrive/colab_SBV2train_sample/SBV2-train.ipynb', recursive=True)[0])
print(drive_path)

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = f"{drive_path}/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = f"{drive_path}/model_assets"

#モデルの保存場所が作成されていなければ作成する
import os
os.makedirs(assets_root, exist_ok=True)

import yaml
with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

# （オプション）ユーザ辞書を指定する

SBV2では、特殊な読み方をする単語などに対して、ユーザ側が読みを指定できる機能がある。

例えば「Aivis」をそのままSBV2に読ませると「エーアイヴイアイエス」と読む。一方で、適切にユーザ辞書ファイル「default.csv」を更新すると「アイヴィス」と読ませることができる。

学習時に、ユーザ辞書を適用させる場合は、Google Driveのユーザ辞書を更新後、`dir_update_FLAG = True`に変更し、下記のセルを実行する。

すると、Google Driveのユーザ辞書csvファイルが、適切な場所にコピーされるため、学習時に反映される


In [4]:
#ユーザ辞書を更新しない場合は、このセルを実行しなくて良い。
from pathlib import Path
import shutil

#ユーザ辞書を更新する場合は、下記のフラグをTrueにする。
dir_update_FLAG = False
default_dict_path = f"{drive_path}/dict_data/default.csv"
copy_dir = f"dict_data"

if os.path.exists(default_dict_path) and dir_update_FLAG:
    shutil.copy(default_dict_path, copy_dir)
    print(f"'{default_dict_path}' を '{copy_dir}' にコピーしました。")


# 学習パラメータを設定する

In [5]:
#実行前に辞書を追加する

# 日本語を学習させる場合は、Trueで良い。他言語を学習させるならFalseにする
use_jp_extra = True

# 基本は4でいい。GPUのメモリに余裕がある場合は、増やすと性能が上がるかも
batch_size = 4

# 30-100程度で十分だと思うが、増やすと性能が高いモデルが存在する確率が上がる
# 今回は学習時間の短縮のために20epochに設定する。
epochs = 20

# 何stepごとにモデルの途中経過を保存するかを指定する。無料のGoogle Driveを利用している場合は、保存容量が少ないため、多めに指定すると良い
# 少なく指定すればするほど、性能の高いモデルが存在する確率が上がり、かつ、不慮の事態で途中で学習が停止した際に、再開することができる
# Google Driveの容量が潤沢にある場合は、200-500程度でも良い
save_every_steps = 1500

#音量の正規化を行うかどうか。基本はFalseで良い
normalize = False

#音声の前後の無音期間をトリミングするかどうか。基本はFalseで良い
trim = False

#読みのエラーが発生した時にどう処理するか。基本はskipで良い
yomi_error = "skip"

# 前処理を実施する

この段階で、テキストベースの書き起こし内容を、音素、アクセント表示に変更する処理や、Bert特徴量などを取得して保存する処理などが実行される。

ユーザ辞書の設定なども、この処理で利用される。
学習を途中から再開する場合は、このセルを実行する必要はない。

下記のエラーが出たら、ランタイムを再起動して、再度最初から実行する。
```
NameError: name '_C' is not defined
```

In [ ]:
#再実行する場合はここは実行しない
#サーバに接続できないエラーが出たら再実行する

from gradio_tabs.train import preprocess_all
from style_bert_vits2.nlp.japanese import pyopenjtalk_worker
from style_bert_vits2.nlp.japanese.user_dict import update_dict


pyopenjtalk_worker.initialize_worker()
update_dict()

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False,
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=1000,
    yomi_error=yomi_error,
)

# 学習に利用する設定ファイルを作成する

学習に利用するパラメータを途中で変更したい場合は、このタイミングで作成される`Data`フォルダのモデル内にある`config.json`を直接編集すれば良い

In [7]:
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
#model_name = "your_model_name"

#事前学習重みを変更する場合はこのタイミングでdata/modelの中のsafetensorを変更する

import yaml
from gradio_tabs.train import get_path

paths = get_path(model_name)
dataset_path = str(paths.dataset_path)
config_path = str(paths.config_path)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

# 学習を開始する

無料版のGoogle Driveを利用している場合は、15GBしか保存容量がないはずなので、定期的にゴミ箱の中の重みファイルを完全削除してください。

1回途中経過が保存されるたびに、1.5GB程度の容量が必要になります。

In [ ]:
if use_jp_extra:
    !python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}
else:
    !python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

# 実行後Colabノードブックのランタイムを削除する

特に、有料版のColabでは、接続時間に応じてコンピューティングユニットが消費されてしまうので、節約のため、学習完了したら即座にColabのランタイムを削除する必要がある。

下記セルにおいて、`FLAG = True`として、実行すると、ランタイムが接続解除される

In [ ]:
# ノートブックの解放
FLAG = False

from google.colab import runtime

if FLAG:
    runtime.unassign()